In [1]:
import tkinter as tk
from tkinter import messagebox
import cv2
import numpy as np
import os

end_restart_window = None  # Define end_restart_window in global scope
usernames = ["admin", "Lotanna", "Desmond"]
pins = ["1111","2222","3333"]

#function to get index
def getIndex(target):
    try:
        index = usernames.index(target)
        return index
    except ValueError:
        return -1

# Function to authenticate username and password
def authenticate(username, pin):
    # Your authentication logic here
    # For example, compare with stored username-password pairs
    index = getIndex(username)
    
    if pins[index]==pin:
        return True
    else:
        return False

# Function to handle login button click
def login_clicked():
    username = username_entry.get()
    password = password_entry.get()
    if authenticate(username, password):
        messagebox.showinfo("Login Successful", "Welcome, " + username)
        login_window.destroy()
        select_video_window()
    else:
        messagebox.showerror("Login Failed", "Invalid username or password")

# Function to select video
def select_video_window():
    select_video_window = tk.Tk()
    select_video_window.title("Select Video")

    # Add padding and styling
    select_video_window.geometry("300x150")
    select_video_window.configure(bg="#f0f0f0")

    # Label
    instruction_label = tk.Label(select_video_window, text="Select a video from the list:", bg="#f0f0f0", fg="#333", font=("Helvetica", 20))
    instruction_label.pack(pady=10)

    # Dropdown menu to select video
    videos = get_video_list()
    selected_video = tk.StringVar(select_video_window)
    selected_video.set(videos[0])  # Default value
    video_dropdown = tk.OptionMenu(select_video_window, selected_video, *videos)
    video_dropdown.pack(pady=10)

    # Button to proceed
    proceed_button = tk.Button(select_video_window, text="Proceed", font=("Helvetica", 12), command=lambda: proceed_with_video(selected_video.get(), select_video_window))
    proceed_button.pack(pady=10)

    select_video_window.mainloop()




# Function to get list of videos
def get_video_list():
    video_folder = "vids"
    videos = [file for file in os.listdir(video_folder) if file.endswith(".mp4")]
    return videos

# Function to proceed with selected video
def proceed_with_video(video_name, select_video_window):
    select_video_window.destroy()
    option_window(video_name)

# Function to ask user for operation choice
def option_window(video_name):
    option_window = tk.Tk()
    option_window.title("Choose Operation")

    # Set background color to white
    option_window.configure(bg="white")

    # Add padding and set window size
    option_window.geometry("400x250")

    # Label
    operation_label = tk.Label(option_window, text="Select Operation:", bg="white", fg="black", font=("Helvetica", 20))
    operation_label.pack(pady=10)

    # Radio buttons for selecting operation
    operation_var = tk.IntVar()
    tk.Radiobutton(option_window, text="Watch Video as is", variable=operation_var, value=1, bg="white", fg="black").pack(anchor="w")
    tk.Radiobutton(option_window, text="Perform Object Detection", variable=operation_var, value=2, bg="white", fg="black").pack(anchor="w")
    tk.Radiobutton(option_window, text="Reduce Noise", variable=operation_var, value=3, bg="white", fg="black").pack(anchor="w")
    tk.Radiobutton(option_window, text="Sharpen Image", variable=operation_var, value=4, bg="white", fg="black").pack(anchor="w")
    tk.Radiobutton(option_window, text="Adjust Brightness and Contrast", variable=operation_var, value=5, bg="white", fg="black").pack(anchor="w")

    # Frame to contain buttons
    button_frame = tk.Frame(option_window, bg="white")
    button_frame.pack(pady=10)

    # Button to proceed
    proceed_button = tk.Button(button_frame, text="Proceed", command=lambda: perform_operation(video_name, operation_var.get(), option_window), bg="white", fg="black")
    proceed_button.pack(side="left", padx=10)

    # Button to go back
    back_button = tk.Button(button_frame, text="Back", command= lambda: back_button_selected(option_window), bg="white", fg="black")
    back_button.pack(side="left", padx=10)

    option_window.mainloop()

def back_button_selected(option_window):
    option_window.destroy()
    select_video_window()


# Function to perform selected operation
def perform_operation(video_name, operation, option_window):
    option_window.destroy()
    video_path = os.path.join("vids", video_name)
    if operation == 1:
        messagebox.showinfo("Message", "Tkinter operations are paused while the video is playing")
        watch_video(video_path)
    elif operation == 2:
        messagebox.showinfo("Message", "Tkinter operations are paused while the object are being detected")
        perform_object_detection(video_path)
    elif operation == 3:
        messagebox.showinfo("Message", "Tkinter operations are paused Performing Noise reduction")
        reduce_noise(video_path)
    elif operation == 4:
        messagebox.showinfo("Message", "Tkinter operations are paused Sharpening video")
        sharpen_video(video_path)
    elif operation == 5:
        messagebox.showinfo("Message", "Tkinter operations are paused Adjusting brightness and contrast")
        adjust_brightness_contrast(video_path)
    else:
        choice = messagebox.askyesno("Prompt", "Nothing was selected. Do you want to end operation?")
        if choice:
            ask_end_restart()
        else:
            select_video_window()




# Function to watch video
def watch_video(video_path):
    cap = cv2.VideoCapture(video_path)
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        cv2.imshow("Video", frame)
        if cv2.waitKey(25) & 0xFF == ord('q'):
            break
    cap.release()
    cv2.destroyAllWindows()

    # Ask user to end or restart operation
    ask_end_restart()

# Function to perform object detection on video
def perform_object_detection(video_path):
    # Load YOLO
    net = cv2.dnn.readNet('cfg/yolov3.weights', 'cfg/yolov3.cfg')
    classes = []
    with open('cfg/coco.names', 'r') as f:
        classes = f.read().splitlines()

    # Open video capture
    cap = cv2.VideoCapture(video_path)

    # Get video properties
    fps = cap.get(cv2.CAP_PROP_FPS)
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

    # Define codec and create VideoWriter object to save the output video
    fourcc = cv2.VideoWriter_fourcc(*'XVID')
    out = cv2.VideoWriter('output_with_detection.avi', fourcc, fps, (width, height))

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        # Preprocess input frame
        blob = cv2.dnn.blobFromImage(frame, 1/255.0, (416, 416), swapRB=True, crop=False)
        net.setInput(blob)

        # Forward pass through the network
        output_layers_names = net.getUnconnectedOutLayersNames()
        layer_outputs = net.forward(output_layers_names)

        # Process detection results
        boxes = []
        confidences = []
        class_ids = []

        for output in layer_outputs:
            for detection in output:
                scores = detection[5:]
                class_id = np.argmax(scores)
                confidence = scores[class_id]
                if confidence > 0.5:
                    center_x = int(detection[0] * width)
                    center_y = int(detection[1] * height)
                    w = int(detection[2] * width)
                    h = int(detection[3] * height)

                    x = int(center_x - w / 2)
                    y = int(center_y - h / 2)

                    boxes.append([x, y, w, h])
                    confidences.append(float(confidence))
                    class_ids.append(class_id)

        # Non-maximum suppression to remove redundant overlapping boxes
        indexes = cv2.dnn.NMSBoxes(boxes, confidences, 0.5, 0.4)

        font = cv2.FONT_HERSHEY_PLAIN
        colors = np.random.uniform(0, 255, size=(len(classes), 3))

        # Draw bounding boxes and labels
        for i in indexes.flatten():
            x, y, w, h = boxes[i]
            label = str(classes[class_ids[i]])
            confidence = str(round(confidences[i], 2))
            color = colors[i]
            cv2.rectangle(frame, (x, y), (x+w, y+h), color, 2)
            cv2.putText(frame, label + " " + confidence, (x, y + 20), font, 2, (255, 255, 255), 2)

        # Write the frame to the output video
        out.write(frame)

        # Display the frame
        cv2.imshow('Frame', frame)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    # Release the video capture and writer objects
    cap.release()
    out.release()
    cv2.destroyAllWindows()
    # Example code
    # messagebox.showinfo("Performing Object Detection", "Object detection in progress...")

    # Ask user to end or restart operation
    ask_end_restart()



# Function to watch video
def play_video(video):
    while video.isOpened():
        ret, frame = video.read()
        if not ret:
            break
        cv2.imshow("Video", frame)
        if cv2.waitKey(25) & 0xFF == ord('q'):
            break
    video.release()
    cv2.destroyAllWindows()

    # Ask user to end or restart operation
    ask_end_restart()


def ask_end_restart():
    global end_restart_window
    end_restart_window = tk.Tk()
    end_restart_window.title("End or Restart Operation")

    # Set background color to white
    end_restart_window.configure(bg="white")

    # Add padding and set window size
    end_restart_window.geometry("300x150")

    # Label
    instruction_label = tk.Label(end_restart_window, text="Do you want to end or restart the operation?", bg="white", fg="black")
    instruction_label.pack(pady=10)

    # Button to end operation
    end_button = tk.Button(end_restart_window, text="End Operation", command=end_operation, bg="white", fg="red")
    end_button.pack(pady=5)

    # Button to restart operation
    restart_button = tk.Button(end_restart_window, text="Restart Operation", command=restart_operation, bg="white", fg="green")
    restart_button.pack(pady=5)

    end_restart_window.mainloop()


# Function to end operation
def end_operation():
    end_restart_window.destroy()

# Function to restart operation
def restart_operation():
    global end_restart_window
    end_restart_window.destroy()
    select_video_window()


#Operations
# Function to reduce noise in the video
def reduce_noise(video_path):
    cap = cv2.VideoCapture(video_path)
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        filtered_image = cv2.medianBlur(frame, 15)
        cv2.imshow('Video with Reduced Noise', filtered_image)
        if cv2.waitKey(25) & 0xFF == ord('q'):
            break
    cap.release()
    cv2.destroyAllWindows()
    ask_end_restart()
    

# Function to sharpen the video
def sharpen_video(video_path):
    cap = cv2.VideoCapture(video_path)
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        kernel = np.array([[0, -1, 0], [-1, 5, -1], [0, -1, 0]])
        sharpened_image = cv2.filter2D(frame, -1, kernel)
        cv2.imshow('Sharpened Video', sharpened_image)
        if cv2.waitKey(25) & 0xFF == ord('q'):
            break
    cap.release()
    cv2.destroyAllWindows()
    ask_end_restart()
    

# Function to adjust brightness and contrast of the video
def adjust_brightness_contrast(video_path):
    cap = cv2.VideoCapture(video_path)
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        brightness = 5
        contrast = 1.5
        adjusted_image = cv2.addWeighted(frame, contrast, np.zeros(frame.shape, frame.dtype), 0, brightness)
        cv2.imshow('Video with Adjusted Brightness and Contrast', adjusted_image)
        if cv2.waitKey(25) & 0xFF == ord('q'):
            break
    cap.release()
    cv2.destroyAllWindows()
    ask_end_restart()
    



# Create the login window
login_window = tk.Tk()
login_window.title("Login")

# Add some padding and styling
login_window.geometry("300x200")
login_window.configure(bg="#f0f0f0")


#Top of Login
open_label = tk.Label(login_window, text="Login", bg="#f0f0f0", fg="#333", font=("Helvetica", 25))
open_label.grid(row=0, column=0,columnspan=2, padx=10, pady=10)

# Username and password fields
username_label = tk.Label(login_window, text="Username:", bg="#f0f0f0", fg="#333", font=("Helvetica", 15))
username_label.grid(row=1, column=0, padx=10, pady=10, sticky="w")
username_entry = tk.Entry(login_window, font=("Helvetica", 12))
username_entry.grid(row=1, column=1, padx=10, pady=10)

password_label = tk.Label(login_window, text="Password:", bg="#f0f0f0", fg="#333", font=("Helvetica", 15))
password_label.grid(row=2, column=0, padx=10, pady=10, sticky="w")
password_entry = tk.Entry(login_window, show="*", font=("Helvetica", 12))
password_entry.grid(row=2, column=1, padx=10, pady=10)

# Login button
login_button = tk.Button(login_window, text="Login", command=login_clicked, bg="white", font=("Helvetica", 12))
login_button.grid(row=3, column=0, columnspan=2, pady=15)

login_window.mainloop()


2024-10-11 07:46:16.009 Python[64023:4917824] WARNING: Secure coding is not enabled for restorable state! Enable secure coding by implementing NSApplicationDelegate.applicationSupportsSecureRestorableState: and returning YES.
